In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import gc
import os

# ==========================
# 0. 설정
# ==========================
SAVE_FILE = "fivb_women_players_stream.csv"
FLUSH_SIZE = 2000   # 이 개수마다 CSV에 저장

# ==========================
# 1. Selenium 설정
# ==========================
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)

driver.get("https://www.fivb.com/players/players-database/")
time.sleep(6)

# ==========================
# 2. Gender → Women 필터
# ==========================
gender_dropdown = wait.until(
    EC.element_to_be_clickable((
        By.XPATH,
        '//*[@id="players-database"]/div[1]/div/div[2]/div[3]/div/div/div/div/div/div'
    ))
)
gender_dropdown.click()
time.sleep(1)

women_btn = wait.until(
    EC.element_to_be_clickable((
        By.XPATH,
        '//*[@id="players-database"]/div[1]/div/div[2]/div[3]/div/div/div/div/ul/li[3]/a'
    ))
)
women_btn.click()
time.sleep(5)

# ==========================
# 3. CSV 준비
# ==========================
fieldnames = [
    "first_name", "last_name", "full_name",
    "country", "team", "gender", "age", "profile_url"
]

file_exists = os.path.exists(SAVE_FILE)
csv_file = open(SAVE_FILE, "a", newline="", encoding="utf-8")
writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

if not file_exists:
    writer.writeheader()

# ==========================
# 4. Load More + 스트리밍 저장
# ==========================
buffer = []
total_saved = 0

while True:
    rows = driver.find_elements(
        By.XPATH,
        '//*[@id="players-database"]/div[2]/div/div[1]/div[2]/div'
    )

    for row in rows[len(buffer):]:
        try:
            gender = row.find_element(By.XPATH, './div/div[5]').text.strip()
            if gender.lower() != "women":
                continue

            try:
                country = row.find_element(By.XPATH, './div/div[1]//img').get_attribute("alt")
            except:
                country = None

            player = {
                "first_name": row.find_element(By.XPATH, './div/div[4]').text.strip(),
                "last_name": row.find_element(By.XPATH, './div/div[3]').text.strip(),
                "full_name": (
                    row.find_element(By.XPATH, './div/div[4]').text.strip() + " " +
                    row.find_element(By.XPATH, './div/div[3]').text.strip()
                ),
                "country": country,
                "team": row.find_element(By.XPATH, './div/div[2]').text.strip(),
                "gender": gender,
                "age": row.find_element(By.XPATH, './div/div[6]').text.strip(),
                "profile_url": row.find_element(By.XPATH, './div/div[7]//a').get_attribute("href")
            }

            buffer.append(player)

            if len(buffer) >= FLUSH_SIZE:
                writer.writerows(buffer)
                total_saved += len(buffer)
                print(f"💾 저장: {total_saved}")
                buffer.clear()
                gc.collect()

        except:
            continue

    try:
        load_more = driver.find_element(
            By.XPATH,
            '//*[@id="players-database"]/div[2]/div/div[2]/div/button'
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", load_more)
        time.sleep(3)
    except:
        print("✅ Load More 종료")
        break

# ==========================
# 5. 마지막 flush
# ==========================
if buffer:
    writer.writerows(buffer)
    total_saved += len(buffer)

csv_file.close()
driver.quit()

print(f"🎉 완료! 총 저장된 여성 선수 수: {total_saved}")


The chromedriver version (142.0.7444.162) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (143.0.7499.41); currently, chromedriver 143.0.7499.42 is recommended for chrome 143.*, so it is advised to delete the driver in PATH and retry


💾 저장: 2000
